In [3]:
# import packages
import pandas as pd

In [5]:
# load data
data_path = "../data/answers-dataset-0.2.0.csv"
df = pd.read_csv(data_path)

In [7]:
df.head()

,id,age,sex,race,ethnicity,orientation,transgender,handedness,education,income,...,subs_can_05to08,subs_can_08to11,subs_can_11to14,subs_can_14to17,subs_can_17to20,subs_can_20to23,subs_can_23to02,subs_can_02to05,subs_can_3,visit
0,2,18.75,Female,White,Non-Hispanic,Heterosexual,No,Right,High school or less,"$25,000 - $50,000",...,No,No,No,No,No,No,No,No,Never,1
1,3,19.74,Female,Multiracial,Non-Hispanic,Bisexual,No,Right,High school or less,Prefer not to say,...,No,No,No,No,No,No,No,No,Never,1
2,4,27.33,Female,White,Non-Hispanic,Bisexual,No,Right,High school or less,"< $25,000",...,No,Yes,No,Yes,No,Yes,No,No,Often,1
3,5,29.60,Female,White,Non-Hispanic,Heterosexual,No,Right,College or more,Prefer not to say,...,No,No,No,No,No,No,No,No,Never,1
4,6,26.71,Female,White,Hispanic,Heterosexual,No,Right,High school or less,"$50,000 - $100,000",...,No,No,No,No,No,No,No,No,Never,1


In [ ]:
# some of the data has already been preprocessed calculating a final score 
# for some variables, e.g. CESD test

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 971 entries, 0 to 970
Columns: 294 entries, id to visit
dtypes: float64(28), int64(178), object(88)
memory usage: 2.2+ MB


In [9]:
df.describe()

,id,age,wd_ttb,wd_tts,wd_bts,wd_sol,wd_nwak,wd_waso,wd_tfa,wd_tob,...,inq_15,inq_thwartedbelong,inq_perceivedburden,mh_meds_1,mh_meds_2,mh_meds_3,mh_meds_4,mh_meds_5,mh_meds_6,visit
count,971.000000,945.000000,971.000000,971.000000,971.000000,971.000000,971.000000,971.000000,971.000000,971.000000,...,971.000000,971.000000,971.000000,971.000000,236.000000,105.000000,39.000000,19.000000,10.000000,971.0
mean,652.503605,20.583704,23.137137,24.178208,63.769310,35.189320,1.713697,8.573347,32.522842,33.242842,...,5.049434,26.430484,9.981462,0.243048,0.444915,0.371429,0.487179,0.526316,0.400000,1.0
std,365.834032,4.105206,1.605166,1.605271,52.839519,34.024028,2.120204,13.391395,1.891405,1.951301,...,1.978135,12.193714,7.021885,0.429145,0.498013,0.485504,0.506370,0.512989,0.516398,0.0
min,2.000000,18.010000,18.000000,18.000000,0.000000,0.000000,0.000000,0.000000,24.960000,27.120000,...,1.000000,9.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,333.000000,18.780000,22.080000,23.040000,30.000000,15.000000,1.000000,1.000000,31.200000,31.920000,...,4.000000,16.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
50%,681.000000,19.360000,23.040000,24.000000,60.000000,30.000000,1.000000,5.000000,32.400000,33.120000,...,6.000000,25.000000,6.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.0
75%,977.500000,20.540000,24.000000,24.960000,90.000000,45.000000,2.000000,10.000000,33.600000,34.560000,...,7.000000,36.000000,11.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0
max,1248.000000,52.690000,30.000000,30.960000,480.000000,240.000000,30.000000,120.000000,40.080000,41.280000,...,7.000000,57.000000,42.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0


In [10]:
colnames = df.columns.tolist()

In [11]:
print(colnames)

['id', 'age', 'sex', 'race', 'ethnicity', 'orientation', 'transgender', 'handedness', 'education', 'income', 'wd_ttb', 'wd_tts', 'wd_bts', 'wd_sol', 'wd_nwak', 'wd_waso', 'wd_tfa', 'wd_tob', 'wd_ema', 'wd_sq', 'wd_tib', 'wd_twt', 'wd_tst', 'wd_se', 'we_ttb', 'we_tts', 'we_bts', 'we_sol', 'we_nwak', 'we_waso', 'we_tfa', 'we_tob', 'we_ema', 'we_sq', 'we_tib', 'we_twt', 'we_tst', 'we_se', 'msw', 'msf', 'msfsc1', 'msfsc2', 'socialjetlag', 'psqi_1', 'psqi_2', 'psqi_3', 'psqi_4', 'psqi_5a', 'psqi_5b', 'psqi_5c', 'psqi_5d', 'psqi_5e', 'psqi_5f', 'psqi_5g', 'psqi_5h', 'psqi_5i', 'psqi_5j', 'psqi_6', 'psqi_7', 'psqi_8', 'psqi_9', 'psqi_10', 'psqi_11a', 'psqi_11b', 'psqi_11c', 'psqi_11d', 'psqi_11e', 'psqi_c1', 'psqi_c2', 'psqi_c3', 'psqi_c4', 'psqi_c5', 'psqi_c6', 'psqi_c7', 'psqi_total', 'brisc_1', 'brisc_2', 'brisc_3', 'brisc_4', 'brisc_total', 'isi_1', 'isi_2', 'isi_3', 'isi_4', 'isi_5', 'isi_6', 'isi_7', 'isi_total', 'upps_1', 'upps_2', 'upps_3', 'upps_4', 'upps_5', 'upps_6', 'upps_7', 'upp

In [12]:
# identify predictor variables
pred_vars = [
    # general demographic
    'id', 'age', 'sex', 'race', 'ethnicity', 'orientation', 'transgender', 
    'handedness', 'education', 'income', 
    # weekday sleep info
    'wd_ttb', 'wd_tts', 'wd_bts', 'wd_sol', 'wd_nwak', 'wd_waso', 'wd_tfa', 
    'wd_tob', 'wd_ema', 'wd_sq', 'wd_tib', 'wd_twt', 'wd_tst', 'wd_se', 
    # weekend sleep info
    'we_ttb', 'we_tts', 'we_bts', 'we_sol', 'we_nwak', 'we_waso', 'we_tfa', 
    'we_tob', 'we_ema', 'we_sq', 'we_tib', 'we_twt', 'we_tst', 'we_se',
    # weekday-weekend comparison info
    'msw', 'msf', 'msfsc1', 'msfsc2', 'socialjetlag',
    # pittsburgh sleep quality index
    'psqi_1', 'psqi_2', 'psqi_3', 'psqi_4', 'psqi_5a', 'psqi_5b', 'psqi_5c', 
    'psqi_5d', 'psqi_5e', 'psqi_5f', 'psqi_5g', 'psqi_5h', 'psqi_5i', 'psqi_5j', 
    'psqi_6', 'psqi_7', 'psqi_8', 'psqi_9', 'psqi_10', 'psqi_11a', 'psqi_11b', 
    'psqi_11c', 'psqi_11d', 'psqi_11e', 'psqi_c1', 'psqi_c2', 'psqi_c3', 'psqi_c4', 
    'psqi_c5', 'psqi_c6', 'psqi_c7', 'psqi_total',
    # brief index of sleep control 
    'brisc_1', 'brisc_2', 'brisc_3', 'brisc_4', 'brisc_total',
    # insomnia severity index
    'isi_1', 'isi_2', 'isi_3', 'isi_4', 'isi_5', 'isi_6', 'isi_7', 'isi_total',
    # sleep disorders symptoms
    'sds_1', 'sds_2', 'sds_3', 'sds_4', 'sds_5', 'sds_6', 'sds_7', 'sds_8', 
    'sds_9', 'sds_10', 'sds_11', 'sds_12', 'sds_13', 'sds_14', 'sds_15', 
    'sds_16', 'sds_17', 'sds_18', 'sds_19', 'sds_20', 'sds_21', 'sds_22', 'sds_23', 
    'sds_24', 'sds_25',
    # disturbing dream and nightmare
    'ddnsi_1', 'ddnsi_2', 'ddnsi_3', 'ddnsi_4', 'ddnsi_5', 'ddnsi_total',
    # caffeine
    'subs_caff_1', 'subs_caff_05to08', 'subs_caff_08to11', 'subs_caff_11to14', 
    'subs_caff_14to17', 'subs_caff_17to20', 'subs_caff_20to23', 'subs_caff_23to02', 
    'subs_caff_02to05',
    # tobacco
    'subs_tob_1', 'subs_tob_05to08', 'subs_tob_08to11', 'subs_tob_11to14', 
    'subs_tob_14to17', 'subs_tob_17to20', 'subs_tob_20to23', 'subs_tob_23to02', 
    'subs_tob_02to05', 'subs_tob_3',
    # alcohol
    'subs_alc_1', 'subs_alc_05to08', 'subs_alc_08to11', 'subs_alc_11to14', 
    'subs_alc_14to17', 'subs_alc_17to20', 'subs_alc_20to23', 'subs_alc_23to02', 
    'subs_alc_02to05', 'subs_alc_3',
    # cannabis
    'subs_can_1', 'subs_can_05to08', 'subs_can_08to11', 'subs_can_11to14', 
    'subs_can_14to17', 'subs_can_17to20', 'subs_can_20to23', 'subs_can_23to02', 
    'subs_can_02to05', 'subs_can_3'
    ]

In [ ]:
# subset_df = df[predictor_variables + ["outcome_1"]]